# Mitiq Zero Noise Extrapolation - Parameter optimizer guide


This notebook serves as a guide to use every feature of the Mitiq Zero Noise Extrapolation (ZNE) parameter optimizer. 

There are 3 optimization methods for ZNE parameters provided and 6 noise models and 7 example circuits included as a proof of concept, but the purpose of this ZNE optimizer is for the user to input their own backend and circuit. Easy "plug and play" functions are provided to optimize over any set of backends and circuits, along with some examples for simplification. 


Run the following code block to import all necessary libraries:

In [1]:
from experiment import make_executor, SILENCE_WARNINGS
from noise_model_backends import get_noise_backend
from circuits import get_circuit
from routines import light_brute_force_search, brute_force_search, adaptive_search

Set the following to "True" to silence some warnings that may appear during experimentation.

In [2]:
SILENCE_WARNINGS=True 

#### Creation of the desired noisy backend and circuit that will be the input of the optimizer

In order to select the desired noise model and circuit, choose the noisy backend from the list and call the function like in the given example.

The following noise models are included: {depolarizing, amplitude_damping, phase_damping, readout, thermal, general_zne}.

In [ ]:
NOISE_MODEL="depolarizing"
backend_1 = get_noise_backend(NOISE_MODEL)

Each backend has associated a set of default parameters that can be changed. See the implementation of each noisy backend to learn more about this. A custom backend can also be provided as it is shown below.

In [4]:
# Some packages have to be installed to run this cell (running '%pip install qiskit-ibm-runtime qiskit-aer' may help)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
backend_2 = AerSimulator.from_backend(FakeSherbrooke())


This next block of code allows to select the circuit and create it. The following circuits are pre defined: {ghz, mirror_circuits, rb_circuits, rotated_rb_circuits, random_clifford_t, w_state, qpe}.

Each circuit has associated a verify_func which processes raw measurement counts to compute the specific success metric of the experiment
and an ideal_result, the noise-free value used as a ground truth baseline to calculate the experimental error.


In [5]:
CIRCUIT="mirror_circuits"
circ_1, verify_func_1, ideal_result_1= get_circuit(CIRCUIT)

A custom circuit can be provided. The function get_circuit must be called to get its associated verify_func and ideal_result. The result  considered as the right one can be also indicated using 'target' parameter. For example, below a GHZ circuit with 5 qubits is manually created and the state '11111' is set as the good result.


In [ ]:
from qiskit import QuantumCircuit
n = 5
ghz_circ = QuantumCircuit(n)
ghz_circ.h(0)
for idx in range(n - 1):
    ghz_circ.cx(idx, idx + 1)

circ_2, verify_func_2, ideal_result_2= get_circuit(ghz_circ, target='11111')
#circ_2, verify_func_2, ideal_result_2= get_circuit(ghz_circ)
print(ideal_result_2) # Since we are only couting '11111' as good result, ideal_result should be 1/2

0.4999999999999999


#### Creation of the executor needed by the optimization routines

In [24]:
exe_1=make_executor(backend_1, verify_func_1, shots=4096)
exe_2=make_executor(backend_2, verify_func_2, shots=4096)


#### Creation of set of test "batch" experiments

The following parameter values can be set: a list of lists of scaling factors, the scaling methods and the extrapolation methods. The set of possible values for these last two parameters are {"global", "local_random", "local_all", "identity_scaling"} and {"linear", "richardson", "polynomial", "exponential", "poly-exp", "adaptive-exp"} respectively. The following batch test is the default one used if the user does not specify one.

In [8]:
zne_batch_test = {
    "noise_scaling_factors": [[1, 2, 3], [1, 3, 5], [1, 5, 9]],
    "noise_scaling_method": ["global", "local_random", "local_all", "identity_scaling"],
    "extrapolation": ["linear", "richardson", "polynomial", "exponential", "poly-exp", "adaptive-exp"],
}

### Optimization routines usage guide

#### Brute force search

This routine evaluates every possible parameter combination within the provided search space. It iterates through all defined noise scaling factors, scaling methods and extrapolation methods to identify the configuration that yields the closest value to the 'ideal_result'. Some configurations might fail to execute successfully, it will be reported at the beginning of the results output.

In [9]:
brute_force_search(circ_1, exe_1, ideal_result_1, zne_batch_test)
#brute_force_search(circ_1, exe_2, ideal_result_1)

7 configurations failed due to extrapolation errors
BEST CONFIGURATION:
{
    "noise_scaling_factors": [1, 3, 5],
    "noise_scaling_method": "global",
    "extrapolation": "poly-exp"
}
Ideal result:      1.0
Best result:       1.0056388397551554
Absolute error:    0.005638839755155445


A file with the top 5 results can be created if a name for it is given.

In [10]:
brute_force_search(circ_2, exe_2, ideal_result_2, results_file="results")

17 configurations failed due to extrapolation errors
BEST CONFIGURATION:
{
    "noise_scaling_factors": [1, 2, 3],
    "noise_scaling_method": "local_random",
    "extrapolation": "polynomial"
}
Ideal result:      0.4999999999999999
Best result:       0.5085449218749998
Absolute error:    0.008544921874999889

File saved: results.csv


In [11]:
import pandas as pd
def show_results(filename):
    df = pd.read_csv(filename)
    print(f"BEST RESULTS")
    print(df.to_string(index=False, formatters={"Ideal":  "{:.2f}".format, "Result": "{:.5f}".format, "Error":  "{:.5f}".format }))
show_results("results.csv")


BEST RESULTS
      Method Extrapolation Scale Factors Ideal  Result   Error
local_random    polynomial     [1, 2, 3]  0.50 0.50854 0.00854
      global  adaptive-exp     [1, 3, 5]  0.50 0.49037 0.00963
      global  adaptive-exp     [1, 2, 3]  0.50 0.51088 0.01088
      global    richardson     [1, 2, 3]  0.50 0.51196 0.01196
   local_all    polynomial     [1, 3, 5]  0.50 0.48590 0.01410


#### Light brute force search

The second method is a lighter version of brute force since it optimizes each parameter locally. In general, the execution time will be lower but the result might not be the optimal one. There is an optional parameter max_iter which is a stopping criterion in the case of getting stuck in an infinite loop during the search. 

In [ ]:
light_brute_force_search(circ_1, exe_1, ideal_result_1, max_iter=10)
#light_brute_force_search(circ_1, exe_1, ideal_result_1)
#light_brute_force_search(circ_1, exe_1, ideal_result_1, zne_batch_test)
#light_brute_force_search(circ_1, exe_1, ideal_result_1, zne_batch_test, results_file="results")

#### Adaptive Search
This method employs an iterative strategy. It starts by finding the best configuration for a base set of scaling factors [1, 3, 5] and then progressively adds higher factors to the list. The algorithm continues to expand the factor list only if the error improves by more than the specified 'tolerance', stopping once the 'max_factors' limit is reached. Both parameters are optional, and their default values are shown in the example below. The stopping criterion is shown at the beginning of the output.

In [25]:
#adaptive_search(circ_2, exe_2, ideal_result_2, tolerance=1e-4, max_factors=10)
#adaptive_search(circ_2, exe_2, ideal_result_2)
#adaptive_search(circ_2, exe_2, ideal_result_2, zne_batch_test)
adaptive_search(circ_2, exe_2, ideal_result_2, results_file="results")

Adaptive search stopped: error worsened
BEST CONFIGURATION:
{
    "noise_scaling_factors": [1, 3, 5],
    "noise_scaling_method": "global",
    "extrapolation": "polynomial"
}
Ideal result:      0.4999999999999999
Best result:       0.4853820800781247
Absolute error:    0.014617919921875167

File saved: results.csv
